# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

In [1]:
# import required libraries
import pandas as pd
import numpy as np

## Example Quantile

In [2]:
example_expenses=[30,40,50,60,80,90,100,120,130]
np.median(example_expenses)
np.quantile(example_expenses,q=0.75, interpolation= 'nearest')

100

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

**Solution:** Import the data from the database. Nothing different from the other parts of the lab apart from the name of the database.

In [3]:
orders = pd.read_csv('Orders.csv',index_col=0)
orders.head(2)

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


**Note:** It's always good practice to take an initial look at the data and see if it needs some cleaning and adjustments. For example we can check if there any NaN values and see if the types of the columns make sense.

In [4]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 397923
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   index         397924 non-null  int64  
 1   InvoiceNo     397924 non-null  int64  
 2   StockCode     397924 non-null  object 
 3   year          397924 non-null  int64  
 4   month         397924 non-null  int64  
 5   day           397924 non-null  int64  
 6   hour          397924 non-null  int64  
 7   Description   397924 non-null  object 
 8   Quantity      397924 non-null  int64  
 9   InvoiceDate   397924 non-null  object 
 10  UnitPrice     397924 non-null  float64
 11  CustomerID    397924 non-null  int64  
 12  Country       397924 non-null  object 
 13  amount_spent  397924 non-null  float64
dtypes: float64(2), int64(8), object(4)
memory usage: 45.5+ MB


As we can see there are no NaN values, since every column has the same number of non-null values as the number of entries: 397924.

The InvoiceDate columns is currently of `object` type, which means that it's a string. We can convert it to datetime so it'll be easier to work with.

Also the `index` column seems kind of redundant since we already have the pandas index. We will drop it.

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

**Solution Sub Problem 1:**  
First of all we need to aggregate the amount_spent for each customer. This can be easily done by grouping the dataframe by `CustomerID` and then using the `sum()` function to aggregate the values.

In [5]:
customers_agg = orders.groupby('CustomerID').sum()
customers_agg.head()

,index,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
12346,37126,541431,2011,1,2,10,74215,1.04,77183.60
12347,30294108,101296926,365971,1383,441,2219,2458,481.21,4310.00
12348,1900064,16869685,62324,257,111,472,2341,178.71,1797.24
12349,26148016,42165457,146803,803,73,657,631,605.10,1757.55
12350,838678,9231629,34187,34,51,272,197,65.30,334.40


Note that we only need the sum of the amount_spent, the other columns are not giving us any useful information since we're just summing Invoice numbers, years, months, etc.

We will therefore drop everything except `amount_spent`. We are using double `[[]]` around the column name so that we will save a DataFrame and not a Series. The only actual difference here is in the way in which a Jupyter notebook prints a DataFrame and a Series: it simply looks nicer in the first case.

In [6]:
customers_agg = customers_agg[['amount_spent']]

In [7]:
customers_agg.head()

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


**Solution Sub Problem 2:**  
We now need to select which customers fall in the quantile ranges that we were given at the start of the exercise, that is 95% and 75%. We can use the `quantile()` method to see where the two quantile falls. Anything higher than the breakpoint is part of that specific quantile. We then select them by filtering the database.

In [8]:
q_75 = customers_agg.quantile(0.75)[0]
q_95 = customers_agg.quantile(0.95)[0]

In [9]:
q_75, q_95

(1661.6400000000003, 5840.181999999983)

**Solution Sub Problem 3:**  
Now that we know which customers are in the 75 and 95 percentile by amount_spent we can simply use that information to apply the relevant labels, that is VIP and Preferred. We can therefore create a new column in the original aggregate DataFrame that will have `VIP` for the 95 percentile and `Preferred` for the 75. There are a lot of ways to do this, but just for the sake of looking at a new function we can use the numpy `select` function, which needs a list of conditions paired with a list of outputs. The doc is here: https://docs.scipy.org/doc/numpy/reference/generated/numpy.select.html

In [10]:
customers_agg['label']=\
customers_agg.amount_spent.apply(lambda x: 'VIP' if x> q_95 else('Preferred' if x>q_75 else None ))
customers_agg.head()


,amount_spent,label
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,None


## Q2: How to identify which country has the most VIP Customers?

**Solution:**  
Now that we have labelled the customers in the `customers_agg` dataframe, which also contains the relevant `CustomerID`, we can simply extract the related information and apply it to the original `orders` dataframe. Then we can group the dataframe by country and see which one has the most by aggregating the data using the `count` method.

One of the fastest way is joining the two dataframes on the `CustomerID` column so that the labels will be automatically applied to the relevant CustomerIDs.

In [11]:
customers_agg['label'].head(2)

CustomerID
12346          VIP
12347    Preferred
Name: label, dtype: object

In [12]:
orders.head(2)

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [13]:
# your code here
orders = orders.join(customers_agg['label'] , on='CustomerID')

In [14]:
orders.head(2)

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,label
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred


In [15]:
orders_vip = orders.loc[orders['label'] == 'VIP']

In [16]:
orders_vip.head(2)

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,label
26,26,536370,22728,2010,12,3,8,alarm clock bakelike pink,24,2010-12-01 08:45:00,3.75,12583,France,90.0,VIP
27,27,536370,22727,2010,12,3,8,alarm clock bakelike red,24,2010-12-01 08:45:00,3.75,12583,France,90.0,VIP


In [17]:
orders_vip.groupby('Country')['CustomerID'].nunique()

Country
Australia            2
Belgium              1
Channel Islands      1
Cyprus               1
Denmark              1
EIRE                 2
Finland              1
France               9
Germany             10
Japan                2
Netherlands          1
Norway               1
Portugal             2
Singapore            1
Spain                2
Sweden               1
Switzerland          3
United Kingdom     177
Name: CustomerID, dtype: int64

As we can see the country with the most VIP customers is the United Kingdom with 84185.

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

**Solutions:**  
The solution is the same as Q2, we simply take all the rows where the label is either VIP or Preferred.

In [18]:
# your code here
orders_vip_pref = orders.loc[(orders['label'] == 'VIP') | (orders['label'] == 'Preferred')]

In [19]:
orders_vip_pref.groupby('Country')['CustomerID'].nunique()

Country
Australia            4
Austria              3
Belgium             12
Canada               1
Channel Islands      4
Cyprus               4
Denmark              3
EIRE                 3
Finland              5
France              29
Germany             39
Greece               1
Iceland              1
Israel               2
Italy                5
Japan                4
Lebanon              1
Malta                1
Netherlands          1
Norway               7
Poland               1
Portugal             7
Singapore            1
Spain                9
Sweden               2
Switzerland          9
United Kingdom     932
Name: CustomerID, dtype: int64

The United Kingdom is still the country with most customers when only taking into account VIP and Preferred ones.